In [1]:
import torch, gpytorch, random
import numpy as np
from gpytorch import settings
from scipy.io import loadmat
from math import floor

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [25]:
def load_uci_data(
    data_dir,
    dataset,
    seed,
    train_pct = 0.8,
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    verbose=False,
):


    set_seed(seed)


    data = torch.Tensor(loadmat(data_dir + dataset + ".mat")["data"])

    X = data[:, :-1]
    y = data[:, -1]

    good_dimensions = X.var(dim=-2) > 1.0e-10
    if int(good_dimensions.sum()) < X.size(1):
        if verbose:
            print(
                "Removed %d dimensions with no variance"
                % (X.size(1) - int(good_dimensions.sum()))
            )
        X = X[:, good_dimensions]

    X_min, X_max = X.min(0)[0], X.max(0)[0]
    X = X - X_min
    X = 2.0 * (X / X_max) - 1.0
    y_mu, y_std = y.mean(), y.std()
    y -= y_mu
    y /= y_std

    shuffled_indices = torch.randperm(X.size(0))
    X = X[shuffled_indices, :]
    y = y[shuffled_indices]

    train_n = int(floor(train_pct * X.size(0)))

    train_x = X[:train_n, :].contiguous().to(device)
    train_y = y[:train_n].contiguous().to(device)

    test_x = X[train_n:, :].contiguous().to(device)
    test_y = y[train_n:].contiguous().to(device)

    if verbose:
        print("Loaded data with input dimension of {}".format(test_x.size(-1)))

    return train_x, train_y, test_x, test_y, [X_min, X_max, y_mu, y_std]
train_x, train_y, test_x, test_y, transforms = load_uci_data("../alternating-projection-for-gp/uci/", "3droad", 0, verbose=True)

Loaded data with input dimension of 3


In [5]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.keops.MaternKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)
model.cuda()
likelihood.cuda();

In [6]:
train_x_small, train_y_small = torch.rand([2000,10], device="cuda:0"), torch.rand([2000], device="cuda:0")
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
likelihood_small = gpytorch.likelihoods.GaussianLikelihood()
model_small = ExactGPModel(train_x_small, train_y_small, likelihood_small)
model_small.cuda()
likelihood_small.cuda();

In [7]:
lt = likelihood(model(train_x)).lazy_covariance_matrix

In [23]:
with gpytorch.settings.cg_tolerance(0.1), gpytorch.settings.max_preconditioner_size(150), gpytorch.settings.use_pivchol_preconditioner():
    lt = likelihood(model(train_x)).lazy_covariance_matrix
    _, plt, _ = lt._preconditioner()

Using Pivoted Cholesky preconditioner


OutOfMemoryError: CUDA out of memory. Tried to allocate 238.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 20.22 GiB is allocated by PyTorch, and 137.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [21]:
plt.shape

torch.Size([347899, 347899])

In [ ]:
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [ ]:
model.likelihood.noise = 0.05
model.likelihood.noise.item()

In [ ]:
with gpytorch.settings.cg_tolerance(0.1), gpytorch.settings.max_preconditioner_size(0), gpytorch.settings.max_nyssvd_preconditioner_size(15):
    gpytorch.settings.record_residual.lst_residual_norm = []
    mll(model(train_x), train_y)

In [ ]:
res_cg = gpytorch.settings.record_residual.lst_residual_norm

In [ ]:
import math
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))
plt.scatter([i for i in range(len(res_cg))], torch.log(torch.tensor(res_cg)), marker='.')

*Some test stuff for linear solves with alt-proj and cj*

In [ ]:
with torch.no_grad(), gpytorch.settings.cg_tolerance(0.1):
    gpytorch.settings.record_residual.lst_residual_norm = []
    sol = gpytorch.utils.alternating_projection(train_x, model.covar_module, 0.01, train_y, 1000)

In [ ]:
res_altproj = gpytorch.settings.record_residual.lst_residual_norm

In [ ]:
from gpytorch.lazy import DiagLazyTensor, AddedDiagLazyTensor
with torch.no_grad(), gpytorch.settings.cg_tolerance(0.1), gpytorch.settings.max_preconditioner_size(500), gpytorch.settings.max_preconditioner_size(0):
    gpytorch.settings.record_residual.lst_residual_norm = []
    lt = AddedDiagLazyTensor(model.covar_module(train_x), DiagLazyTensor(torch.tensor([0.01 for i in range(len(train_y))]).cuda()))
    precon_closure = lt._preconditioner()[0]
    gpytorch.utils.linear_cg(lt.matmul, train_y, preconditioner=precon_closure, max_iter = 100)

In [ ]:
res_cg = gpytorch.settings.record_residual.lst_residual_norm

In [ ]:
import math
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))
plt.scatter([i for i in range(len(res_altproj))], torch.log(torch.tensor(res_altproj)), marker='.')
plt.scatter([i for i in range(len(res_cg))], torch.log(torch.tensor(res_cg)), marker='.')

RPCholesky trial

In [ ]:
N, k = 1000, 1000
A = torch.rand([N,N], device='cuda:0')
A = 0.1 * A.T @ A + 0.01 * torch.eye(N, device='cuda:0')

In [ ]:
diags = A.diag()

##G is the skinny factor transposed
G = torch.zeros([k,N], device='cuda:0')
arr_idx = []

for i in range(k):
    idx = torch.multinomial(diags/diags.sum(), 1)
    arr_idx.append(idx)
    G[i,:] = (A[idx,:] - G[:i,idx].T @ G[:i,:]) / torch.sqrt(diags[idx])
    diags -= G[i,:]**2
    diags = diags.clip(min=0)

In [ ]:
lt._lazy_tensor._approx_diag()